# Example 6: Multiple risk measures

This example shows how to use multiple risk measures.

In [1]:
using PortfolioOptimisers, PrettyTables
# Format for pretty tables.
tsfmt = (v, i, j) -> begin
    if j == 1
        return Date(v)
    else
        return v
    end
end;
resfmt = (v, i, j) -> begin
    if j == 1
        return v
    else
        return isa(v, Number) ? "$(round(v*100, digits=3)) %" : v
    end
end;
mipresfmt = (v, i, j) -> begin
    if j ∈ (1, 2, 3)
        return v
    else
        return isa(v, Number) ? "$(round(v*100, digits=3)) %" : v
    end
end;

## 1. ReturnsResult data

We will use the same data as the previous example.

In [2]:
using CSV, TimeSeries, DataFrames

X = TimeArray(CSV.File(joinpath(@__DIR__, "SP500.csv.gz")); timestamp = :Date)[(end - 252):end]
pretty_table(X[(end - 5):end]; formatters = [tsfmt])

# Compute the returns
rd = prices_to_returns(X)

┌────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────
│  timestamp │    AAPL │     AMD │     BAC │     BBY │     CVX │      GE │     ⋯
│ Dates.Date │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Flo ⋯
├────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────
│ 2022-12-20 │ 131.916 │   65.05 │  31.729 │  77.371 │ 169.497 │  62.604 │ 310 ⋯
│ 2022-12-21 │ 135.057 │   67.68 │  32.212 │  78.729 │  171.49 │   64.67 │ 314 ⋯
│ 2022-12-22 │ 131.846 │   63.86 │  31.927 │  78.563 │ 168.918 │  63.727 │ 311 ⋯
│ 2022-12-23 │ 131.477 │   64.52 │  32.005 │  79.432 │  174.14 │  63.742 │ 314 ⋯
│ 2022-12-27 │ 129.652 │   63.27 │  32.065 │   79.93 │ 176.329 │  64.561 │ 314 ⋯
│ 2022-12-28 │ 125.674 │   62.57 │  32.301 │  78.279 │ 173.728 │  63.883 │  31 ⋯
└────────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴──────
                                                              14 columns omitted


ReturnsResult
    nx ┼ 20-element Vector{String}
     X ┼ 252×20 Matrix{Float64}
    nf ┼ nothing
     F ┼ nothing
    ts ┼ 252-element Vector{Dates.Date}
    iv ┼ nothing
  ivpa ┴ nothing


## 2. Preparatory steps

We'll provide a vector of continuous solvers as a failsafe.

In [3]:
using Clarabel
slv = [Solver(; name = :clarabel1, solver = Clarabel.Optimizer,
              settings = Dict("verbose" => false),
              check_sol = (; allow_local = true, allow_almost = true)),
       Solver(; name = :clarabel3, solver = Clarabel.Optimizer,
              settings = Dict("verbose" => false, "max_step_fraction" => 0.9),
              check_sol = (; allow_local = true, allow_almost = true)),
       Solver(; name = :clarabel5, solver = Clarabel.Optimizer,
              settings = Dict("verbose" => false, "max_step_fraction" => 0.8),
              check_sol = (; allow_local = true, allow_almost = true)),
       Solver(; name = :clarabel7, solver = Clarabel.Optimizer,
              settings = Dict("verbose" => false, "max_step_fraction" => 0.70),
              check_sol = (; allow_local = true, allow_almost = true))];

## 3. Multiple risk measures

### 3.1 Equally weighted sum

Some risk measures can use precomputed prior statistics which take precedence over the ones in `PriorResult`. We can make use of this to minimise the variance with different covariance matrices simultaneously.

We will also precompute the prior statistics to minimise redundant work. First lets create a vector of Variances onto which we will push the different variances. We'll use 5 variance estimators, and their equally weighted sum.

  1. Denoised covariance using the spectral algorithm.
  2. Gerber 1 covariance.
  3. Smyth Broby 1 covariance.
  4. Mutual Information covariance.
  5. Distance covariance.
  6. Equally weighted sum of all the above covariances.

For the multi risk measure optimisation, we will weigh each risk measure equally. It should give the same result as adding all covariances together, but not the same as averaging the weights of the individual optimisations.

In [4]:
pr = prior(HighOrderPriorEstimator(), rd.X)

ces = [PortfolioOptimisersCovariance(;
                                     mp = DenoiseDetoneAlgMatrixProcessing(;
                                                                           dn = Denoise(;
                                                                                        alg = SpectralDenoise()))),
       PortfolioOptimisersCovariance(; ce = GerberCovariance()),
       PortfolioOptimisersCovariance(; ce = SmythBrobyCovariance(; alg = SmythBroby1())),
       PortfolioOptimisersCovariance(; ce = MutualInfoCovariance()),
       PortfolioOptimisersCovariance(; ce = DistanceCovariance())]

5-element Vector{PortfolioOptimisersCovariance}:
 PortfolioOptimisersCovariance
  ce ┼ Covariance
     │    me ┼ SimpleExpectedReturns
     │       │   w ┴ nothing
     │    ce ┼ GeneralCovariance
     │       │   ce ┼ SimpleCovariance: SimpleCovariance(true)
     │       │    w ┴ nothing
     │   alg ┴ Full()
  mp ┼ DenoiseDetoneAlgMatrixProcessing
     │     pdm ┼ Posdef
     │         │      alg ┼ UnionAll: NearestCorrelationMatrix.Newton
     │         │   kwargs ┴ @NamedTuple{}: NamedTuple()
     │      dn ┼ Denoise
     │         │      alg ┼ SpectralDenoise()
     │         │     args ┼ Tuple{}: ()
     │         │   kwargs ┼ @NamedTuple{}: NamedTuple()
     │         │   kernel ┼ typeof(AverageShiftedHistograms.Kernels.gaussian): AverageShiftedHistograms.Kernels.gaussian
     │         │        m ┼ Int64: 10
     │         │        n ┼ Int64: 1000
     │         │      pdm ┼ Posdef
     │         │          │      alg ┼ UnionAll: NearestCorrelationMatrix.Newton
     │         │

Lets define a vector of variance risk measure using each of the different covariance matrices.

In [5]:
rs = [Variance(; sigma = cov(ce, rd.X)) for ce in ces]
all_sigmas = zeros(length(rd.nx), length(rd.nx))
for r in rs
    all_sigmas .+= r.sigma
end
push!(rs, Variance(; sigma = all_sigmas))

6-element Vector{Variance{RiskMeasureSettings{Float64, Nothing, Bool}, Matrix{Float64}, Nothing, Nothing, SquaredSOCRiskExpr}}:
 Variance
  settings ┼ RiskMeasureSettings
           │   scale ┼ Float64: 1.0
           │      ub ┼ nothing
           │     rke ┴ Bool: true
     sigma ┼ 20×20 Matrix{Float64}
      chol ┼ nothing
        rc ┼ nothing
       alg ┴ SquaredSOCRiskExpr()

 Variance
  settings ┼ RiskMeasureSettings
           │   scale ┼ Float64: 1.0
           │      ub ┼ nothing
           │     rke ┴ Bool: true
     sigma ┼ 20×20 Matrix{Float64}
      chol ┼ nothing
        rc ┼ nothing
       alg ┴ SquaredSOCRiskExpr()

 Variance
  settings ┼ RiskMeasureSettings
           │   scale ┼ Float64: 1.0
           │      ub ┼ nothing
           │     rke ┴ Bool: true
     sigma ┼ 20×20 Matrix{Float64}
      chol ┼ nothing
        rc ┼ nothing
       alg ┴ SquaredSOCRiskExpr()

 Variance
  settings ┼ RiskMeasureSettings
           │   scale ┼ Float64: 1.0
           │      ub ┼ no

We'll minimise the variance for each individual risk measure and then we'll minimise the equally weighted sum of all risk measures.

In [6]:
results = [optimise(MeanRisk(; r = r, opt = JuMPOptimiser(; pr = pr, slv = slv)))
           for r in rs]
mean_w = zeros(length(results[1].w))
for res in results[1:5]
    mean_w .+= res.w
end
mean_w ./= 5
res = optimise(MeanRisk(; r = rs, opt = JuMPOptimiser(; pr = pr, slv = slv)))
pretty_table(DataFrame(:assets => rd.nx, :denoise => results[1].w, :gerber1 => results[2].w,
                       :smyth_broby1 => results[3].w, :mutual_info => results[4].w,
                       :distance => results[5].w, :mean_w => mean_w,
                       :sum_covs => results[6].w, :multi_risk => res.w);
             formatters = [resfmt])

┌────────┬──────────┬──────────┬──────────────┬─────────────┬──────────┬────────
│ assets │  denoise │  gerber1 │ smyth_broby1 │ mutual_info │ distance │   mea ⋯
│ String │  Float64 │  Float64 │      Float64 │     Float64 │  Float64 │  Floa ⋯
├────────┼──────────┼──────────┼──────────────┼─────────────┼──────────┼────────
│   AAPL │    0.0 % │    0.0 % │        0.0 % │     1.263 % │    0.0 % │  0.25 ⋯
│    AMD │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    BAC │    0.0 % │  2.988 % │        0.0 % │     2.166 % │  2.279 % │  1.48 ⋯
│    BBY │    0.0 % │    0.0 % │        0.0 % │      0.74 % │    0.0 % │  0.14 ⋯
│    CVX │ 17.488 % │  9.462 % │     15.048 % │     4.007 % │  9.961 % │ 11.19 ⋯
│     GE │    0.0 % │  2.287 % │        0.0 % │     2.702 % │  4.347 % │  1.86 ⋯
│     HD │    0.0 % │    0.0 % │        0.0 % │     2.713 % │  3.792 % │  1.30 ⋯
│    JNJ │ 76.031 % │ 23.934 % │     56.706 % │    17.458 % │  17.28 % │ 38.28 ⋯
│    JPM │    0.0 % │    0.0

For extra credit we can do the same but maximising the ratio of return to risk.

In [7]:
results = [optimise(MeanRisk(; r = r, obj = MaximumRatio(),
                             opt = JuMPOptimiser(; pr = pr, slv = slv))) for r in rs]
mean_w = zeros(length(results[1].w))
for res in results[1:5]
    mean_w .+= res.w
end
mean_w ./= 5
res = optimise(MeanRisk(; r = rs, obj = MaximumRatio(),
                        opt = JuMPOptimiser(; pr = pr, slv = slv)))

pretty_table(DataFrame(:assets => rd.nx, :denoise => results[1].w, :gerber1 => results[2].w,
                       :smyth_broby1 => results[3].w, :mutual_info => results[4].w,
                       :distance => results[5].w, :mean_w => mean_w,
                       :sum_covs => results[6].w, :multi_risk => res.w);
             formatters = [resfmt])

┌────────┬──────────┬──────────┬──────────────┬─────────────┬──────────┬────────
│ assets │  denoise │  gerber1 │ smyth_broby1 │ mutual_info │ distance │   mea ⋯
│ String │  Float64 │  Float64 │      Float64 │     Float64 │  Float64 │  Floa ⋯
├────────┼──────────┼──────────┼──────────────┼─────────────┼──────────┼────────
│   AAPL │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    AMD │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    BAC │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    BBY │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    CVX │    0.0 % │  3.321 % │        0.0 % │     9.888 % │    0.0 % │  2.64 ⋯
│     GE │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│     HD │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    JNJ │    0.0 % │    0.0 % │        0.0 % │       0.0 % │    0.0 % │    0. ⋯
│    JPM │    0.0 % │    0.0

### 3.2 Different weights and scalarisers

All optimisations accept multiple risk measures in the same way. We can also provide different weights for each measure and four different scalarisers, `SumScalariser`, `MaxScalariser`, `LogSumExpScalariser` which work for all optimisation estimators, and `MinScalariser` which only works for hierarchical ones.

For clustering optimisations, the scalarisers apply to each sub-optimisation, so what may be the choice of risk to "minimise" for one cluster may not be the minimal risk for others, or the overall portfolio. This inconsistency is unavoidable but should not be a problem in practice as the point of hierarchical optimisations is not to provide the absolute minimum risk, but a good trade-off between risk and diversification.

It is also possible to mix any and all compatible risk measures. We will demonstrate this by mixing the variance with the negative skewness.

In this example we have tuned the weight of the negative skewness to demonstrate how clusters may end up with different risk measures due to the choice of scalariser.

We will use the heirarchical equal risk contribution optimisation, precomputing the clustering results using the direct bubble hierarchy tree algorithm.

The [`HierarchicalEqualRiskContribution`]-(@ref) optimisation estimator accepts inner and outer risk measures and inner and outer scalarisers.

In [8]:
clr = clusterise(ClustersEstimator(; alg = DBHT()), pr.X)
r = [Variance(), NegativeSkewness(; settings = RiskMeasureSettings(; scale = 0.1))]

results = [optimise(HierarchicalEqualRiskContribution(; ri = r[1],# inner (intra-cluster) risk measure
                                                      ro = r[1],# outer (inter-cluster) risk measure
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r[2], ro = r[2],
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,#
                                                      scai = SumScalariser(),# inner (intra-cluster)
                                                      scao = SumScalariser(),# outer (inter-cluster)
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = MaxScalariser(),
                                                      scao = MaxScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = MinScalariser(),
                                                      scao = MinScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = LogSumExpScalariser(),
                                                      scao = LogSumExpScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr)))]

pretty_table(DataFrame(:assets => rd.nx, :variance => results[1].w,
                       :neg_skew => results[2].w, :sum_sca => results[3].w,
                       :max_sca => results[4].w, :min_sca => results[5].w,
                       :log_sum_exp => results[6].w); formatters = [resfmt])

┌────────┬──────────┬──────────┬──────────┬─────────┬──────────┬─────────────┐
│ assets │ variance │ neg_skew │  sum_sca │ max_sca │  min_sca │ log_sum_exp │
│ String │  Float64 │  Float64 │  Float64 │ Float64 │  Float64 │     Float64 │
├────────┼──────────┼──────────┼──────────┼─────────┼──────────┼─────────────┤
│   AAPL │  1.847 % │  4.367 % │  2.949 % │ 3.286 % │  2.571 % │     3.152 % │
│    AMD │  0.627 % │    2.3 % │  1.056 % │ 1.115 % │  1.354 % │     1.069 % │
│    BAC │  2.221 % │  6.575 % │  3.636 % │  3.95 % │  3.871 % │     3.789 % │
│    BBY │  1.138 % │  2.166 % │  1.781 % │ 2.024 % │  1.275 % │     1.942 % │
│    CVX │  4.525 % │   4.41 % │  5.338 % │ 6.436 % │  3.246 % │    11.606 % │
│     GE │  1.924 % │  2.356 % │  2.923 % │ 3.423 % │  1.387 % │     3.284 % │
│     HD │  2.386 % │  2.995 % │  3.629 % │ 4.244 % │  1.763 % │     4.071 % │
│    JNJ │ 10.746 % │  7.487 % │ 10.587 % │ 7.821 % │ 10.413 % │     6.708 % │
│    JPM │  2.623 % │  5.682 % │  4.153 % │ 4.666 % 

When the weights are different enough that one risk measure domintes over the other in all contexts, then the results of the max and min scalarisers will be as expected, i.e. they will be as if only one risk measure was used.

In [9]:
r = [Variance(), NegativeSkewness()]

results = [optimise(HierarchicalEqualRiskContribution(; ri = r[1],# inner (intra-cluster) risk measure
                                                      ro = r[1],# outer (inter-cluster) risk measure
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r[2], ro = r[2],
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,#
                                                      scai = SumScalariser(),# inner (intra-cluster)
                                                      scao = SumScalariser(),# outer (inter-cluster)
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = MaxScalariser(),
                                                      scao = MaxScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = MinScalariser(),
                                                      scao = MinScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr))),
           optimise(HierarchicalEqualRiskContribution(; ri = r, ro = r,
                                                      scai = LogSumExpScalariser(),
                                                      scao = LogSumExpScalariser(),
                                                      opt = HierarchicalOptimiser(; pe = pr,
                                                                                  cle = clr)))]

pretty_table(DataFrame(:assets => rd.nx, :variance => results[1].w,
                       :neg_skew => results[2].w, :sum_sca => results[3].w,
                       :max_sca => results[4].w, :min_sca => results[5].w,
                       :log_sum_exp => results[6].w); formatters = [resfmt])

┌────────┬──────────┬──────────┬─────────┬─────────┬──────────┬─────────────┐
│ assets │ variance │ neg_skew │ sum_sca │ max_sca │  min_sca │ log_sum_exp │
│ String │  Float64 │  Float64 │ Float64 │ Float64 │  Float64 │     Float64 │
├────────┼──────────┼──────────┼─────────┼─────────┼──────────┼─────────────┤
│   AAPL │  1.847 % │  4.367 % │ 3.946 % │ 4.367 % │  1.847 % │     3.155 % │
│    AMD │  0.627 % │    2.3 % │  1.73 % │   2.3 % │  0.627 % │      1.07 % │
│    BAC │  2.221 % │  6.575 % │ 5.377 % │ 6.575 % │  2.221 % │     3.792 % │
│    BBY │  1.138 % │  2.166 % │ 2.181 % │ 2.166 % │  1.138 % │     1.943 % │
│    CVX │  4.525 % │   4.41 % │ 4.959 % │  4.41 % │  4.525 % │    11.589 % │
│     GE │  1.924 % │  2.356 % │ 3.063 % │ 2.356 % │  1.924 % │     3.286 % │
│     HD │  2.386 % │  2.995 % │ 3.833 % │ 2.995 % │  2.386 % │     4.074 % │
│    JNJ │ 10.746 % │  7.487 % │ 8.942 % │ 7.487 % │ 10.746 % │     6.714 % │
│    JPM │  2.623 % │  5.682 % │ 5.356 % │ 5.682 % │  2.623 % │ 

Note how the max scalariser produced the same weights as the negative skewness and the min scalariser produced the same weights as the variance. This is because in all cases, the same the value of the negative skewness was greater than that of the variance. A similar behaviour can be observed with other clustering optimisers. [`NearOptimalCentering`]-(@ref) can also have unintuitive behaviour when computing the risk bounds with an effective frontier `MaxScalariser` and `MinScalariser` due to the fact that each point in the efficient frontier can have a different risk measure dominating the others.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*